# Optional Ports

In [1]:
import sys
from pathlib import Path

WORKSPACE = Path("..")

sys.path.insert(0, str(WORKSPACE / "src"))

In [2]:
from vcd import VCDWriter

from seqlogic import Module, Vec, run, sleep

In [3]:
class Adder(Module):

    full: bool = False

    def build(self):
        # Ports
        s = self.output(name="s", dtype=Vec[1])
        co = self.output(name="co", dtype=Vec[1])

        a = self.input(name="a", dtype=Vec[1])
        b = self.input(name="b", dtype=Vec[1])

        # Only full adder requires carry-in
        if self.full:
            ci = self.input(name="ci", dtype=Vec[1])

        # Combinational Logic
        if self.full:
            self.expr(s, a ^ b ^ ci)
            self.expr(co, a & b | ci & (a | b))
        else:
            self.expr(s, a ^ b)
            self.expr(co, a & b)

In [4]:
VALUES = [
    ("1b0", "1b0", "1b0"),
    ("1b0", "1b0", "1b1"),
    ("1b0", "1b1", "1b0"),
    ("1b0", "1b1", "1b1"),
    ("1b1", "1b0", "1b0"),
    ("1b1", "1b0", "1b1"),
    ("1b1", "1b1", "1b0"),
    ("1b1", "1b1", "1b1"),
]


class Top(Module):
    """Top Level Module."""

    def build(self):
        a = self.input(name="a", dtype=Vec[1])
        b = self.input(name="b", dtype=Vec[1])
        ci = self.input(name="ci", dtype=Vec[1])

        fa_s = self.logic(name="fa_s", dtype=Vec[1])
        fa_co = self.logic(name="fa_co", dtype=Vec[1])

        ha_s = self.logic(name="ha_s", dtype=Vec[1])
        ha_co = self.logic(name="ha_co", dtype=Vec[1])

        # Design Under Test
        self.submod(
            name="fa",
            mod=Adder.parameterize(full=True),
        ).connect(
            s=fa_s,
            co=fa_co,
            a=a,
            b=b,
            ci=ci,
        )

        self.submod(
            name="ha",
            mod=Adder.parameterize(full=False),
        ).connect(
            s=ha_s,
            co=ha_co,
            a=a,
            b=b,
        )

        self.drv(self.drv_inputs())

    async def drv_inputs(self):
        await sleep(1)

        for a, b, ci in VALUES:
            self.a.next = a
            self.b.next = b
            self.ci.next = ci

            await sleep(1)

        self.a.next = "1b0"
        self.b.next = "1b0"
        self.ci.next = "1b0"
        await sleep(1)

In [5]:
with (
    open("adder.vcd", "w") as f,
    VCDWriter(f, timescale="1ns") as vcdw,
):
    top = Top(name="top")
    top.dump_vcd(vcdw, ".*")
    run(top.elab(), ticks=100)